In [1]:
from xgboost import XGBClassifier
import lightgbm as lgb
from catboost import CatBoostClassifier

import matplotlib.pyplot as plt
import os
import pandas as pd
import numpy as np
import joblib
from datetime import datetime
import sys

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score

sys.path.append(os.path.abspath("D:\\burtm\\Visual_studio_code\\PD_related_projects"))


In [35]:
source_path = 'D:\\burtm\\Visual_studio_code\\PD_related_projects\\outputs\\preprocessed_data\\'
#input_file='icdar_train_df_KAGGLE_20250514_181737.csv'
input_file=source_path+'icdar_EXTRACTED_train_df_dresnet50_20250520_143808.csv'
#icdar_EXTRACTED_train_df_crnn_vgg16_bn_20250520_165111.csv padding
# icdar_EXTRACTED_train_df_crnn_vgg16_bn_20250520_161925.csv crop
# icdar_EXTRACTED_train_df_crnn_vgg16_bn_20250520_153435.csv resizing 
#icdar_EXTRACTED_train_df_dresnet50_20250520_143808.csv 
train_FE = pd.read_csv(input_file)

In [3]:
is_kaggle = False
with_pca=False

In [4]:
# Count numeric columns
num_numeric_cols = train_FE.select_dtypes(include=['number']).shape[1]

# Count columns with missing values
num_missing_cols = train_FE.isnull().any().sum()

# Display results
print(f'Number of numeric columns: {num_numeric_cols}')
print(f'Number of columns with missing values: {num_missing_cols}')

Number of numeric columns: 523
Number of columns with missing values: 0


In [10]:
train_FE.head()

,writer,isEng,same_text,file_name,male,train,index,x,y,x2,...,f759,f760,f761,f762,f763,f764,f765,f766,f767,f768
0,1,0,0,D:\download\PD project\datasets\ICDAR 2013 - G...,0,1,0,0,493,493,...,-0.157444,-0.071830,0.153469,-0.085469,-0.005282,-0.051098,0.051322,-0.049213,0.045018,-0.157976
1,1,0,0,D:\download\PD project\datasets\ICDAR 2013 - G...,0,1,1,493,493,986,...,-0.071231,-0.048658,0.237796,-0.149107,0.064615,-0.074437,0.204165,0.000632,-0.025162,-0.199461
2,1,0,0,D:\download\PD project\datasets\ICDAR 2013 - G...,0,1,2,1479,493,1972,...,-0.123792,-0.072135,0.173672,-0.139977,0.003667,-0.058085,0.085976,0.031146,-0.085116,-0.176518
3,1,0,0,D:\download\PD project\datasets\ICDAR 2013 - G...,0,1,3,986,493,1479,...,-0.169500,-0.034012,0.210294,-0.128820,-0.058314,-0.134394,0.076376,-0.011624,0.096393,-0.211085
4,1,0,0,D:\download\PD project\datasets\ICDAR 2013 - G...,0,1,4,1972,493,2465,...,-0.133709,-0.068476,0.155843,-0.123356,-0.037430,-0.114230,-0.004338,-0.086928,-0.083811,-0.154938


In [36]:
target_label='male'
#target_label='isEng'
if is_kaggle:
    cols_to_drop = ['writer', 'same_text', 'train','page_id','isEng','train','index','male']
else:
    cols_to_drop = [c for c in train_FE.columns if not(c.startswith('f') and len(c) > 1 and c[1].isdigit())]
    print(cols_to_drop)
X_train = train_FE[train_FE['train']==1].drop(columns=cols_to_drop)
y_train = train_FE[train_FE['train']==1][target_label]
X_val = train_FE[train_FE['train']==0].drop(columns=cols_to_drop)
y_val = train_FE[train_FE['train']==0][target_label]

['writer', 'isEng', 'same_text', 'file_name', 'male', 'train', 'index', 'x', 'y', 'x2', 'y2', 'n_cc']


In [37]:
# Define the model
gbm_classifier = GradientBoostingClassifier(
    n_estimators=100,  
    learning_rate=0.1,  
    max_depth=3,  
    random_state=42
)

cat = CatBoostClassifier(iterations=100, learning_rate=0.1, depth=6, verbose=0, random_state=42)

lgbm = lgb.LGBMClassifier(n_estimators=100, learning_rate=0.1, max_depth=-1, random_state=42)

xgb = XGBClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)

feature_extraction_model = {'gbm': gbm_classifier, 'cat': cat, 'lgbm': lgbm, 'xgb': xgb}

In [7]:
selected_model = 'gbm'

In [38]:
import time

#feature_extraction_model = {'gbm': gbm_classifier, 'cat': cat, 'lgbm': lgbm, 'xgb': xgb}
# Define the pipeline
if with_pca:
    from sklearn.decomposition import PCA
    pca = PCA(n_components=384)  
    pipeline = Pipeline([
        ('scaler', StandardScaler()),  # Normalize features
        ('pca', pca),  # Apply PCA
        (selected_model, feature_extraction_model[selected_model])  # Train GBM classifier
    ])
else:    
    pipeline = Pipeline([
        ('scaler', StandardScaler()),  # Normalize features
        (selected_model, feature_extraction_model[selected_model])  # Train GBM classifier
    ])

In [39]:
# Measure the start time
start_time = time.time()

# Fit the model on training data
pipeline.fit(X_train, y_train)

# Measure the end time
end_time = time.time()

# Calculate the time taken
time_taken = end_time - start_time
print(f"Time taken to fit the model: {time_taken:.2f} seconds")

Time taken to fit the model: 674.21 seconds


In [40]:
# Predict on validation data
y_pred = pipeline.predict(X_val)

# Evaluate accuracy
accuracy = accuracy_score(y_val, y_pred)
print(f'Validation Accuracy: {accuracy:.4f}')

Validation Accuracy: 0.6276


In [41]:
# Predict on train data
y_pred = pipeline.predict(X_train)

# Evaluate accuracy
training_accuracy = accuracy_score(y_train, y_pred)
print(f'training Accuracy: {training_accuracy:.4f}')

training Accuracy: 0.8421


In [42]:
source_path = "D:\\burtm\\Visual_studio_code\\PD_related_projects"
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_dir = os.path.join(source_path, "outputs", "models")
os.makedirs(output_dir, exist_ok=True)  # Create the directory if it doesn't exist

if is_kaggle:
    tipo='kaggle'
else:
    tipo='deep'
# Save the model
model_filename = os.path.join(output_dir, f"{tipo}_feature_extraction_model_{target_label}_{selected_model}_val{str(round(accuracy, 2)).replace('.', '-')}_{timestamp}.joblib")
joblib.dump(pipeline, model_filename)
print(f"Model saved to {model_filename}")

Model saved to D:\burtm\Visual_studio_code\PD_related_projects\outputs\models\deep_feature_extraction_model_male_gbm_val0-63_20250520_173200.joblib


In [43]:
# Example usage:
LOG_FILE = output_dir+"\\file_metadata_log.json"
print(f"Log file path: {LOG_FILE}")
print(f"Output file path: {model_filename}")
file_IO.add_or_update_file(
    model_filename, LOG_FILE,
    custom_metadata={
        #"seed": seed,
        "trained on file": os.path.basename(input_file),
        "model": selected_model,
        "model_params": feature_extraction_model[selected_model].get_params(),
        "training time": time_taken,
        "accuracy train": training_accuracy,
        "accuracy val": accuracy,
        "problem type": target_label,
        "input file": input_file,
        "with PCA": with_pca,
        "description": '''I am testing a drnet50 model with a GBM classifier on the features extracted from the images.''' 
    }
)

Log file path: D:\burtm\Visual_studio_code\PD_related_projects\outputs\models\file_metadata_log.json
Output file path: D:\burtm\Visual_studio_code\PD_related_projects\outputs\models\deep_feature_extraction_model_male_gbm_val0-63_20250520_173200.joblib
Updated log for deep_feature_extraction_model_male_gbm_val0-63_20250520_173200.joblib


In [20]:
file_IO.read_metadata(
    model_filename,
    log_path=LOG_FILE
)

Metadata for deep_feature_extraction_model_male_gbm_val0-67_20250517_160824.joblib:
full_path: D:\burtm\Visual_studio_code\PD_related_projects\outputs\models\deep_feature_extraction_model_male_gbm_val0-67_20250517_160824.joblib
size_bytes: 221717
created: 2025-05-17T16:08:25.016231
modified: 2025-05-17T16:08:25.364292
accessed: 2025-05-17T16:08:25.364292
trained on file: icdar_EXTRACTED_train_df_vit-base-patch16-224-in21k_20250517_151642.csv
model: gbm
model_params: {'ccp_alpha': 0.0, 'criterion': 'friedman_mse', 'init': None, 'learning_rate': 0.1, 'loss': 'log_loss', 'max_depth': 3, 'max_features': None, 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 100, 'n_iter_no_change': None, 'random_state': 42, 'subsample': 1.0, 'tol': 0.0001, 'validation_fraction': 0.1, 'verbose': 0, 'warm_start': False}
training time: 277.32010316848755
accuracy train: 0.8478260869565217
accuracy val: 0.66551

# easy access

In [14]:
def reload_modules():
    import importlib
    import utils.file_IO as file_IO
    

    importlib.reload(file_IO)

    return file_IO

file_IO = reload_modules()